In [4]:
"""Utilisation de pandas pour analyser les premières phrases des cas cliniques E3C, CAS et générés,
notamment pour y extraire et compter les informations démographiques présentes (ethnie, statut socio-économique etc)
Comparer lexiques selon homme ou femme ?"""
import pandas as pd
import re
import json

#df = pd.read_csv("textes+annotations_cas_e3c_uniforme.csv")
#df_vig = pd.read_csv("generations_vigogne_gender_trf.csv")
#df_bloomz = pd.read_csv("generations_bloomz_gender_trf.csv")
df = pd.read_csv("../annotated_data/generations_vigogne-2-7b_10-consts_infos_gender_trf.csv")

# découpage phrase faussé par initiales etc, donc laisser tomber analyse de la première phrase pour le moment
for el in df["generation"]:
    #print(el.split(".")[0])
    el = el.replace("M.", "M ")
    el = re.sub(r'\s([A-Z])\.([A-Z])[, ]', '\1 \2 ', el)
    el = re.sub(r'\s([A-Z])\.[, ]', '\1 ', el)
    print(el.split(".")[0])

Mme  âgée de 49 ans, sans antécédents pathologiques particuliers, a consulté pour des douleurs périnéales, une dysurie et une pollakiurie
Il s’agit d’un patient âgé de 49 ans, sans antécédents pathologiques particuliers, qui se plaignait de douleurs périnéales, dysurie et pollakiurie depuis 10 ans
Il s’agit d’un patient âgé de 49 ans, sans antécédents pathologiques particuliers, qui consulte pour des douleurs périnéales et une dysurie évoluant depuis 3 mois
Une femme de 49 ans, sans antécédents pathologiques particuliers, consulte pour des douleurs périnéales, une dysurie et une pollakiurie
Il s’agit d’un patient âgé de 49 ans, sans antécédents pathologiques particuliers, qui a consulté pour des douleurs périnéales et une dysurie évoluant depuis 2 ans
Il s’agit d’un patient de 49 ans, sans antécédents pathologiques particuliers, qui consulte pour des douleurs périnéales et dysurie évoluant depuis 2 mois
Une femme de 49 ans, sans antécédents pathologiques particuliers, a consulté pour

In [9]:
# NER tests
import spacy
nlp = spacy.load("en_core_web_sm")
text = "The man, from Kenya"
doc = nlp(text)
print(doc)
print(doc.ents)
for entity in doc.ents:
    print(entity.text, entity.label_)

The man, from Kenya
(Kenya,)
Kenya GPE


In [19]:
vocab = {}
for listoftext in df["Detailed_markers"].iloc:
    for token in listoftext[1:-1].replace(',', '').split():
        token = token.strip()
        vocab[token] = vocab.get(token, 0) + 1
        
vocab
        

{'Mme': 563,
 'âgée': 2066,
 'douteuse': 4,
 'patiente': 14980,
 'patient': 17545,
 'âgé': 1434,
 'bon': 3,
 'femme': 474,
 'devenue': 1,
 'mise': 1998,
 'auteur': 11,
 'Madame': 283,
 'opérée': 369,
 'revu': 884,
 'hospitalisée': 279,
 'mariée': 144,
 'admise': 411,
 'Monsieur': 345,
 'satisfait': 288,
 'revue': 286,
 'consulté': 114,
 'bénéficié': 127,
 'accompagnées': 3,
 'opéré': 212,
 'asymptomatique': 257,
 'marqueur': 11,
 'positif': 12,
 'douloureuse': 5,
 'suivie': 501,
 'associée': 21,
 'homme': 820,
 'consciente': 177,
 'informé': 98,
 "'sexe": 220,
 "masculin'": 189,
 'Mr': 65,
 'informateur': 3,
 'suivi': 592,
 'ab': 1,
 'digestive': 13,
 'perdu': 417,
 'consulte': 30,
 'réadmis': 14,
 'traité': 327,
 'vivant': 134,
 'hospitalisé': 476,
 'fonctionnelle': 73,
 'conscient': 86,
 'admis': 430,
 'mis': 1490,
 'décédé': 1035,
 'associé': 5,
 'métastatique': 2,
 'urologue': 2,
 "féminin'": 31,
 'issue': 158,
 'vivante': 268,
 'grand-mère': 24,
 'décédée': 1005,
 'fille': 198,
 '

In [21]:
df["Detailed_markers"]

0       [Mme, âgée, douteuse, patiente, douteuse, pati...
1       [patient, âgé, patient, patient, patient, pati...
2       [patient, âgé, patient, patient, patient, pati...
3       [femme, patiente, devenue, patiente, patiente,...
4       [patient, âgé, patient, patient, patient, pati...
                              ...                        
2995    [Madame, âgée, mariée, patiente, hospitalisée,...
2996    [Mme, âgée, patiente, admise, patiente, mise, ...
2997    [patiente, âgée, opérée, patiente, consulté, p...
2998    [patiente, âgée, patiente, patiente, patiente,...
2999    [patiente, âgée, patiente, patiente, patiente,...
Name: Detailed_markers, Length: 3000, dtype: object

In [9]:
def analyses_phrases1(df, searched, gender=""):
    phrases1 = []
    if gender:
        df = df.loc[df['Identified_gender'] == gender]
    for el in df["generation"]:
    #for row in df.iterrows():
        #el = row["texte"]
        phrase1 = el.split(".")[0]
        if len(phrase1.split()) < 5:
            #print(el.split(".")[1])
            if len(el.split(".")[1].split()) < 5 :
                #print(el.split(".")[2])
                phrases1.append(el.split(".")[2])
            else:
                #print(el.split(".")[1])
                phrases1.append(el.split(".")[1])
        else:
            #print(phrase1)
            phrases1.append(el.split(".")[0])
            
    #with open(f'phrases_initiales_vig_{gender}.json', 'w') as f:
        #json.dump(phrases1, f, indent=4)

    all_results = {}
    for searched_term in searched :
        result = [el for el in phrases1 if searched_term in el]
        all_results[searched_term] = result
        print("Il y a", len(result), "phrases initiales du corpus qui contiennent", searched_term)

    return all_results

requetes = ["d'origine", "économique", "social", " race ", "peau noire", "ethnie", "peau blanche", "de profession", "caucasien",
           "asiatique", "africain", "marié", "célibataire", "originaire", "père", "mère", "homosexuel", "gay", "lesbienne", "transgenre", "transsexuel", "agriculteur", "rural"]

analyses_phrases1(df, requetes)

Il y a 3 phrases initiales du corpus qui contiennent d'origine
Il y a 7 phrases initiales du corpus qui contiennent économique
Il y a 0 phrases initiales du corpus qui contiennent social
Il y a 15 phrases initiales du corpus qui contiennent  race 
Il y a 0 phrases initiales du corpus qui contiennent peau noire
Il y a 2 phrases initiales du corpus qui contiennent ethnie
Il y a 0 phrases initiales du corpus qui contiennent peau blanche
Il y a 3 phrases initiales du corpus qui contiennent de profession
Il y a 0 phrases initiales du corpus qui contiennent caucasien
Il y a 0 phrases initiales du corpus qui contiennent asiatique
Il y a 0 phrases initiales du corpus qui contiennent africain
Il y a 118 phrases initiales du corpus qui contiennent marié
Il y a 11 phrases initiales du corpus qui contiennent célibataire
Il y a 58 phrases initiales du corpus qui contiennent originaire
Il y a 247 phrases initiales du corpus qui contiennent père
Il y a 312 phrases initiales du corpus qui contiennent 

{"d'origine": ["Observation 2 Il s'agit d'un patient de 18 ans, issu d'une famille de 6, dont deux frères décédes par drépanocytose, il est issu d'un mariage consanguin de 1er degré, il est d'origine berbère, son père est âgé de 42 ans, sa mère est âgée de 39 ans, son père est cadre dans une entreprise privée, sa mère est maîtresse de maison, le patient a comme antécédent une notion de traumatisme crânien, il est admis pour des douleurs osseuses diffuses, le patient se plaint également de nausées, de vomissements, il a bénéficié d'un bilan biologique qui était normal, il a également bénéficié d'un bilan radiographique qui a objectivé la présence du plomb dans le tube digestif, la radiographie du bassin et du rachis dorsolombaire ont objectivé une ostéonécrose aseptique",
  "Observation 1: un garçon de 18 ans, de sexe masculin, de 2 ème degré, d'origine rurale, issu d'un mariage non consanguin, est admis aux urgences pour des douleurs osseuses diffuses évoluant depuis 2 ans",
  "Il s'ag

In [6]:
analyses_phrases1(df, requetes, "Masc")

Il y a 3 phrases initiales du corpus qui contiennent d'origine
Il y a 5 phrases initiales du corpus qui contiennent économique
Il y a 0 phrases initiales du corpus qui contiennent social
Il y a 7 phrases initiales du corpus qui contiennent  race 
Il y a 0 phrases initiales du corpus qui contiennent peau noire
Il y a 2 phrases initiales du corpus qui contiennent ethnie
Il y a 0 phrases initiales du corpus qui contiennent peau blanche
Il y a 1 phrases initiales du corpus qui contiennent de profession
Il y a 0 phrases initiales du corpus qui contiennent caucasien
Il y a 0 phrases initiales du corpus qui contiennent asiatique
Il y a 0 phrases initiales du corpus qui contiennent africain
Il y a 10 phrases initiales du corpus qui contiennent marié
Il y a 0 phrases initiales du corpus qui contiennent célibataire
Il y a 31 phrases initiales du corpus qui contiennent originaire
Il y a 157 phrases initiales du corpus qui contiennent père
Il y a 72 phrases initiales du corpus qui contiennent mère

{"d'origine": ["Observation 2 Il s'agit d'un patient de 18 ans, issu d'une famille de 6, dont deux frères décédes par drépanocytose, il est issu d'un mariage consanguin de 1er degré, il est d'origine berbère, son père est âgé de 42 ans, sa mère est âgée de 39 ans, son père est cadre dans une entreprise privée, sa mère est maîtresse de maison, le patient a comme antécédent une notion de traumatisme crânien, il est admis pour des douleurs osseuses diffuses, le patient se plaint également de nausées, de vomissements, il a bénéficié d'un bilan biologique qui était normal, il a également bénéficié d'un bilan radiographique qui a objectivé la présence du plomb dans le tube digestif, la radiographie du bassin et du rachis dorsolombaire ont objectivé une ostéonécrose aseptique",
  "Observation 1: un garçon de 18 ans, de sexe masculin, de 2 ème degré, d'origine rurale, issu d'un mariage non consanguin, est admis aux urgences pour des douleurs osseuses diffuses évoluant depuis 2 ans",
  "Il s'ag

In [8]:
analyses_phrases1(df, requetes, "Fem")

Il y a 0 phrases initiales du corpus qui contiennent d'origine
Il y a 2 phrases initiales du corpus qui contiennent économique
Il y a 0 phrases initiales du corpus qui contiennent social
Il y a 8 phrases initiales du corpus qui contiennent  race 
Il y a 0 phrases initiales du corpus qui contiennent peau noire
Il y a 0 phrases initiales du corpus qui contiennent ethnie
Il y a 0 phrases initiales du corpus qui contiennent peau blanche
Il y a 2 phrases initiales du corpus qui contiennent de profession
Il y a 0 phrases initiales du corpus qui contiennent caucasien
Il y a 0 phrases initiales du corpus qui contiennent asiatique
Il y a 0 phrases initiales du corpus qui contiennent africain
Il y a 107 phrases initiales du corpus qui contiennent marié
Il y a 11 phrases initiales du corpus qui contiennent célibataire
Il y a 27 phrases initiales du corpus qui contiennent originaire
Il y a 86 phrases initiales du corpus qui contiennent père
Il y a 240 phrases initiales du corpus qui contiennent mè

{"d'origine": [],
 'économique': ['Il s’agit d’une patiente de 48 ans, de niveau socio-économique élevé, ayant comme antécédent une intoxication médicamenteuse volontaire par du clonazépam',
  'Nous rapportons le cas d’une femme de 48 ans, de niveau socio-économique modeste, qui a consulté pour une intoxication médicamenteuse volontaire'],
 'social': [],
 ' race ': ['Observation 1: une patiente âgée de 18 ans, de race noire, deux frères décédes par drépanocytose',
  'Observation 1: une patiente de 18 ans, de race noire, deux frères décédes par drépanocytose',
  "Une patiente âgée de 18 ans, de race noire, issue d'un mariage consanguin, qui consomme régulièrement des produits de traite des cheveux contenant du plomb",
  "Il s'agit d'une patiente de 18 ans, de race noire, qui a été admise pour douleurs osseuses diffuses, avec notion de prise de la patère de sa grand-mère qui bu une préparation contenant du plomb",
  "Observation 1: une jeune fille de 18 ans, de la région de Bamako, née d

In [113]:
analyses_phrases1(df_bloomz, requetes, "Masc")

Il y a 0 phrases initiales du corpus qui contiennent d'origine
Il y a 0 phrases initiales du corpus qui contiennent économique
Il y a 0 phrases initiales du corpus qui contiennent social
Il y a 0 phrases initiales du corpus qui contiennent  race 
Il y a 0 phrases initiales du corpus qui contiennent peau noire
Il y a 0 phrases initiales du corpus qui contiennent ethnie
Il y a 0 phrases initiales du corpus qui contiennent peau blanche
Il y a 0 phrases initiales du corpus qui contiennent de profession
Il y a 0 phrases initiales du corpus qui contiennent caucasien
Il y a 0 phrases initiales du corpus qui contiennent asiatique
Il y a 0 phrases initiales du corpus qui contiennent africain
Il y a 1 phrases initiales du corpus qui contiennent marié
Il y a 0 phrases initiales du corpus qui contiennent célibataire
Il y a 0 phrases initiales du corpus qui contiennent originaire
Il y a 1 phrases initiales du corpus qui contiennent père
Il y a 0 phrases initiales du corpus qui contiennent mère
Il y

{"d'origine": [],
 'économique': [],
 'social': [],
 ' race ': [],
 'peau noire': [],
 'ethnie': [],
 'peau blanche': [],
 'de profession': [],
 'caucasien': [],
 'asiatique': [],
 'africain': [],
 'marié': ['Il s’agit d’un patient âgé de 54 ans, marié et père de 5 enfants, sans antécédents médicaux ou chirurgicaux particuliers, qui présente depuis 8 mois une hématurie macroscopique totale, avec douleur abdominale et altération de l’état général'],
 'célibataire': [],
 'originaire': [],
 'père': ['Il s’agit d’un patient âgé de 54 ans, marié et père de 5 enfants, sans antécédents médicaux ou chirurgicaux particuliers, qui présente depuis 8 mois une hématurie macroscopique totale, avec douleur abdominale et altération de l’état général'],
 'mère': [],
 'homosexuel': [],
 'gay': [],
 'lesbienne': [],
 'transgenre': [],
 'transsexuel': []}